<h1 align="center"><font color="yellow">SpanMarker for Named Entity Recognition</font></h1>

<font color="yellow">Data Scientist.: PhD.Eddy Giusepe Chirinos Isidro</font>

Links de estudo:

* [SpanMarker](https://tomaarsen.github.io/SpanMarkerNER/notebooks/spacy_integration.html#Using-SpanMarker-models-for-Named-Entity-Recognition-with-spaCy)

* [tomaarsen/SpanMarkerNER](https://github.com/tomaarsen/SpanMarkerNER/tree/main)


# Usando SpanMarker com spaCy

O [SpanMarker](https://github.com/tomaarsen/SpanMarkerNER/tree/main) é um módulo `Python` acessível, mas poderoso, para treinar modelos de Reconhecimento de Entidade Nomeada.

Neste breve caderno, veremos como usar modelos `SpanMarker` pré-treinados com [spaCy](https://spacy.io/).

# Configuração

Primeiro de tudo, ambos `spacy` e o módulo Python `span_marker` precisam ser instalados. Depois, precisamos instalar um `modelo spacy` também. Vamos escolher o mais simples por enquanto: `pt_core_news_sm` (pode usar, também, para Inglês: `en_core_web_sm`).


```
!pip install span-marker==1.2.3   # Eu instalei deste jeito --> https://pypi.org/project/span-marker/1.2.3/

%pip install span_marker spacy

!python -m spacy download pt_core_news_sm

!python -m spacy download en_core_web_sm
```

In [ ]:
#!pip install span-marker==1.2.3

#!pip install spacy

#!python -m spacy download pt_core_news_sm

#!python -m spacy download en_core_web_sm

#!python -m spacy download pt_core_news_lg

# Usando spaCy para Reconhecimento de Entidade Nomeada

Começaremos usando puramente `spaCy` para `NER`, para ajudar a dar uma indicação das alterações que precisam ser feitas para usar os modelos `SpanMarker` para `NER`.


In [18]:
import spacy


# Carregamos o Modelo spaCy
nlp = spacy.load("pt_core_news_lg") # "en_core_web_sm" # "pt_core_news_sm"

# Alimente algum texto através do modelo para obter um doc spaCy
text = """O Eddy é do Perú e a sua esposa Karina é do Brasil. Atualmente eles moram en Vitória-ES. \
Ele trabalha atualmente na empresa CentralIT Tecnologia como Cientista de Dados. O Eddy estudo na UFES o curso de \
Física. Ele nasceu no dia 4 de setembro de 1981 (04/09/1981).
"""
doc = nlp(text)

# E olhe para as Entidades:
doc.ents



(Eddy,
 Perú,
 Karina,
 Brasil,
 Vitória-ES,
 CentralIT Tecnologia como Cientista de Dados,
 Eddy,
 UFES,
 Física)

<font color="orange">O módulo `spaCy` vem com um visualizador conveniente que podemos usar para inspecionar essas entidades de uma maneira mais conveniente, vamos usá-lo.</font>

In [19]:
from spacy import displacy

displacy.render(doc, jupyter=True, style="ent")

Não é exatamente o ideal. Este modelo `spaCy` erra.

# Usando modelos SpanMarker para reconhecimento de entidade nomeada com spaCy

<font color="orange">Podemos facilmente adicionar um modelo `SpanMarker` como um substituto do `pipeline spaCy NER` original. É tão simples quanto uma linha de código.</font>

In [20]:
nlp.add_pipe("span_marker", config={"model": "tomaarsen/span-marker-roberta-large-ontonotes5"})

# pode usar --> "tomaarsen/span-marker-xlm-roberta-base-fewnerd-fine-super"
# pode usar --> "tomaarsen/span-marker-xlm-roberta-large-conllpp-doc-context"

#nlp.add_pipe("span_marker", config={"model": "tomaarsen/span-marker-roberta-large-ontonotes5"})


O modelo de configuração refere-se a [tomaarsen/span-marker-roberta-large-ontonotes5](https://huggingface.co/tomaarsen/span-marker-roberta-large-ontonotes5), um modelo treinado no `OntoNotes v5.0`, o mesmo conjunto de dados usado pelo `pipeline spaCy NER original`. A [spaCy integration API reference](https://tomaarsen.github.io/SpanMarkerNER/api/span_marker.spacy_integration.html) tem mais documentação sobre as opções de configuração. Vamos experimentar o `pipeline spaCy atualizado`.

In [21]:
# Tudo o que precisamos fazer é processar o texto usando o pipeline spaCy atualizado:
doc = nlp(text)

print(doc.ents)

displacy.render(doc, jupyter=True, style="ent")

(Eddy, Perú, Karina, Brasil, Vitória-ES, CentralIT Tecnologia, Eddy, UFES, 04/09/1981)


Muito melhor!

Mas, e se não quisermos usar um modelo com essas etiquetas? Bem, essa integração funciona para qualquer [SpanMarker model on the Hugging Face Hub](https://huggingface.co/models?library=span-marker), então podemos escolher outro. Vamos agora também garantir que o modelo permaneça na `CPU`, apenas para ver como isso funciona.

In [22]:
nlp.remove_pipe("span_marker")

nlp.add_pipe(
    "span_marker",
    config={
        "model": "tomaarsen/span-marker-xlm-roberta-base-fewnerd-fine-super",
        "device": "cpu",
    },
)

doc = nlp(text)
print(doc.ents)
displacy.render(doc, jupyter=True, style="ent")


(Eddy, Perú, Karina, Brasil, Vitória-ES, CentralIT Tecnologia, Eddy, UFES)


# Resumo

<font color="orange">Para resumir, usar `SpanMarker` com `spaCy` é tão simples quanto isto:</font>

In [23]:
import spacy

nlp = spacy.load("pt_core_news_lg", disable=["ner"])
#nlp.add_pipe("span_marker")
nlp.add_pipe("span_marker",
             config={"model": "tomaarsen/span-marker-xlm-roberta-large-conllpp-doc-context", "device": "cpu"},
            )

#text = "Amelia Earhart flew her single engine Lockheed Vega 5B across the Atlantic to Paris."
text = """O Eddy é do Perú e a sua esposa Karina é do Brasil. Atualmente eles moram en Vitória-ES. \
Ele trabalha atualmente na empresa CentralIT Tecnologia como Cientista de Dados. O Eddy estudo na UFES o curso de \
Física. Ele nasceu no dia 4 de setembro de 1981.
"""
doc = nlp(text)

[(entity, entity.label_) for entity in doc.ents]

[(Eddy, 'PER'),
 (Perú, 'LOC'),
 (Karina, 'PER'),
 (Brasil, 'LOC'),
 (Vitória-ES, 'LOC'),
 (CentralIT Tecnologia, 'ORG'),
 (Eddy, 'PER'),
 (UFES, 'ORG')]